## basic use

the next cells show how the %cache magic should be used


Note: This notebook requires the the packages scikit-learn, numpy and cache_magic to be installed

In [1]:
import cache_magic

# delete everthing currently cached
%cache --reset 

# store a new value for a
%cache a = "111"

# fetch the cached value for a
%cache a = "111"

%cache magic is now registered in ipython
resetting entire cache
creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000463


In [2]:
# an examle for an actual use-case
import cache_magic
import numpy as np
from sklearn import svm

%cache --reset

%timeit -n 1 -r 5 %cache -v 1 clf = svm.LinearSVC().fit(np.random.randint(5, size=(5000, 40)), np.random.randint(5, size=(5000)))


resetting entire cache
creating new value for variable 'clf'
loading cached value for variable 'clf'. Time since pickling  0:00:00.000672
loading cached value for variable 'clf'. Time since pickling  0:00:00.001523
loading cached value for variable 'clf'. Time since pickling  0:00:00.002196
loading cached value for variable 'clf'. Time since pickling  0:00:00.002644
The slowest run took 5657.97 times longer than the fastest. This could mean that an intermediate result is being cached.
626 ms ± 1.25 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [3]:
# the following 4 cases use the same version

%cache -r

# without explicit version, the expression (=right hand site of assignment) is used as version
%cache a = 0

# if parameter is an integer, it will be the version
%cache -v 0 a = 1 

# if parameter is a variable name, it's value is used as version
my_version = 0
%cache -v my_version a = 1

# new and old version are converted into a string before comparing them
my_version_2 = "0"
%cache -v my_version_2 a = 1

resetting entire cache
creating new value for variable 'a'
Warning! Expression has changed since last save, which was at 2018-01-25 15:06:27.536005
To store a new value, change the version ('-v' or '--version')  
loading cached value for variable 'a'. Time since pickling  0:00:00.000930
Warning! Expression has changed since last save, which was at 2018-01-25 15:06:27.536005
To store a new value, change the version ('-v' or '--version')  
loading cached value for variable 'a'. Time since pickling  0:00:00.001622
Warning! Expression has changed since last save, which was at 2018-01-25 15:06:27.536005
To store a new value, change the version ('-v' or '--version')  
loading cached value for variable 'a'. Time since pickling  0:00:00.002148


In [4]:
# show everything, that is cached
%cache

# generate some variables
%cache b=3
def fun(x):
    return x+1
%cache c = fun(b)
%cache -v c d = fun(1.1)

# show the new cache
%cache

var name,size(byte),stored at date,version,expression(hash)
a,5,2018-01-25 15:06:27.536005,0,0


creating new value for variable 'b'
creating new value for variable 'c'
creating new value for variable 'd'


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:27.664712,3,3
a,5,2018-01-25 15:06:27.536005,0,0
c,5,2018-01-25 15:06:27.666673,fun(b),fun(b)
d,12,2018-01-25 15:06:27.668137,4,fun(1.1)


## power use

the next cells show how the %cache magic can be used

In [5]:
import cache_magic
import numpy as np
from sklearn import svm
%cache --reset

# even if the expression changes, but not the version, the old value will still be loaded 
# in which case there will be a warning
%cache -v 1 clf = svm.LinearSVC().fit(np.random.randint(5, size=(1000, 40)), np.random.randint(5, size=(1000)))
%cache -v 1 clf = "not a classifier"
print(clf.predict(np.random.randint(5,size=(1,40)))[0])

# without an expression, it will always try to reload the cached value
del clf
%cache -v 1 clf
print(clf.predict(np.random.randint(5,size=(1,40)))[0])

# you can store the current value of a var without an actual statement by assigning it to itself
clf="not a classifier"
%cache -v 2 clf=clf
print(clf)


resetting entire cache
creating new value for variable 'clf'
Warning! Expression has changed since last save, which was at 2018-01-25 15:06:28.366362
To store a new value, change the version ('-v' or '--version')  
loading cached value for variable 'clf'. Time since pickling  0:00:00.000725
4
loading cached value for variable 'clf'. Time since pickling  0:00:00.001897
1
creating new value for variable 'clf'
not a classifier


In [6]:
# while the cache still exists in the file system, the cell can be executed alone
import cache_magic

%cache clf
print(clf)


loading cached value for variable 'clf'. Time since pickling  0:00:00.005745
not a classifier


In [7]:
# the cache is stored in the directory where the kernel was first started in 

import cache_magic
import os

%cache -r
%cache a=1
%cache b=1
%cache c=1

%cache
for root, dirs, files in os.walk(".cache_magic"):
    # there is one folder per cache variable
    print(root)

# if the working dir changes, the .cache-dir stays where it is
%cd ..
%cache
for root, dirs, files in os.walk(".cache_magic"):
    # no output, because no .cache-dir
    print(root)
    

%cd -
%cache
for root, dirs, files in os.walk(".cache_magic"):
    # now we see the cache directory againg
    print(root)
       

resetting entire cache
creating new value for variable 'a'
creating new value for variable 'b'
creating new value for variable 'c'


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.437203,1,1
c,5,2018-01-25 15:06:28.438906,1,1


.cache_magic
.cache_magic/b
.cache_magic/a
.cache_magic/c
/home/jovyan


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.437203,1,1
c,5,2018-01-25 15:06:28.438906,1,1


/home/jovyan/magic


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.437203,1,1
c,5,2018-01-25 15:06:28.438906,1,1


.cache_magic
.cache_magic/b
.cache_magic/a
.cache_magic/c


In [8]:
# always store a new value and never read from cache
%cache -r a=1

# remove a single variable from cache
%cache -r a

# Error:
%cache a

# load last value if possible, and store new value on miss
%cache a = a


resetting cached values for a
creating new value for variable 'a'
resetting cached values for a
Error: variable 'a' not in cache
creating new value for variable 'a'


In [9]:
# load last value if possible, but don't store new value on miss
import cache_magic

del a
%cache a

loading cached value for variable 'a'. Time since pickling  0:00:00.073952


In [10]:
# You can use this magic-module as a regular python module

from cache_magic import CacheCall
cache = CacheCall(get_ipython().kernel.shell)

# setting all parameter by name
cache(
    version="*",
    reset=False,
    var_name="aaa",
    var_value="1+1",
    show_all=False,
    set_debug=True)

# setting all parameter by ordering
cache("1",False,"bbb","1+1",False, False)

# setting parameter selectivly
cache(show_all=True)


creating new value for variable 'aaa'
creating new value for variable 'bbb'


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.565980,a,a
aaa,5,2018-01-25 15:06:28.704483,1+1,1+1
c,5,2018-01-25 15:06:28.438906,1,1
bbb,5,2018-01-25 15:06:28.706382,1,1+1


## development tests

the next cells show how the %cache magic should not be used

these examples are for debug-purposes only

In [11]:
#testing successfull calls

import cache_magic

# Dev-Note: use reload, so you don't have to restart the kernel everytime you change the 
from imp import reload
reload(cache_magic)

my_version = 3
%cache --reset

print("   exptecting: new values")
%cache -v 2 a = "ex3"
%cache -v my_version c = "ex3"
%cache --version my_version sadsda = "ex3"
%cache -v 3 a=""
%cache -v 3 -r a=""
%cache -v 3 -r a=""


print("   exptecting: warnings")
%cache -v 3 a= " _ "
%cache -v 3 sadsda = "ex4"

print("   exptecting: stored values")
%cache -v my_version sadsda = "ex3"
%cache -v 3 sadsda = "ex3"


%cache magic is now registered in ipython
resetting entire cache
   exptecting: new values
creating new value for variable 'a'
creating new value for variable 'c'
creating new value for variable 'sadsda'
creating new value for variable 'a'
resetting cached values for a
creating new value for variable 'a'
resetting cached values for a
creating new value for variable 'a'
   exptecting: warnings
Warning! Expression has changed since last save, which was at 2018-01-25 15:06:28.796198
To store a new value, change the version ('-v' or '--version')  
loading cached value for variable 'a'. Time since pickling  0:00:00.000741
Warning! Expression has changed since last save, which was at 2018-01-25 15:06:28.792367
To store a new value, change the version ('-v' or '--version')  
loading cached value for variable 'sadsda'. Time since pickling  0:00:00.005230
   exptecting: stored values
loading cached value for variable 'sadsda'. Time since pickling  0:00:00.005763
loading cached value for variabl

In [12]:
# testing errors

import cache_magic
reload(cache_magic)

%cache -v "a" a = "ex3"
%cache -v a 1=a

%cache magic is now registered in ipython
Version: "a"
version_param.isdigit(): False
Error: Invalid version. It must either be an Integer, *, or the name of a variable
Error: statement is no valid python: 1=a
 Error: can't assign to literal (<unknown>, line 1)


In [13]:
# testing loading without storing

import cache_magic
reload(cache_magic)

%cache --reset 
a=1
del a

# error: 
%cache a

%cache a=1
del a
%cache a
del a

# error:
%cache -v '1' a

# error
%cache -v 1 a

%cache --reset 
a=1
del a

# Error
%cache -v 0 a

%cache -v * a=1

# Error:
%cache -v * a

%cache magic is now registered in ipython
resetting entire cache
Error: variable 'a' not in cache
creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000447
Version: '1'
version_param.isdigit(): False
Error: Invalid version. It must either be an Integer, *, or the name of a variable
Error: Forced version '1' could not be found, instead found version '1'.If you don't care about a specific version, leave out the version parameter. 
resetting entire cache
Error: variable 'a' not in cache
creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000385


In [14]:

%cache -v "1" a


Version: "1"
version_param.isdigit(): False
Error: Invalid version. It must either be an Integer, *, or the name of a variable


In [15]:

%cache -v 213 a = "1"
# get stored version via error message
%cache -v * a

# testing debug flag '-d'
%cache -d -v 1 -r a = "1"
%cache -d a = "1"


creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000334
resetting cached values for a
creating new value for variable 'a'
resetting because version mismatch
creating new value for variable 'a'


In [16]:
import cache_magic
from imp import reload
reload(cache_magic)

%cache -r a=1
%cache -r a
%cache -r a=1

%cache magic is now registered in ipython
resetting cached values for a
creating new value for variable 'a'
resetting cached values for a
resetting cached values for a
creating new value for variable 'a'


In [17]:
import cache_magic
from imp import reload
reload(cache_magic)

%cache -r a = 1
%cache a
print (a)

%cache magic is now registered in ipython
resetting cached values for a
creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000310
1


In [18]:
import cache_magic
from imp import reload
reload(cache_magic)

def foo(x):
    return x+1

%cache --reset
%cache -v * a= foo(3)
%cache -v * a= foo(3)
%cache -v * a= 2


%cache magic is now registered in ipython
resetting entire cache
creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000325
creating new value for variable 'a'


In [19]:
#!pip install -e .